# Getting started `musif`

`musif` is a Python library to analyze music scores. It is a tool to massively extract features from MusicXML and MuseScore files.

`musif` was born in the context of the [ERC Project "DIDONE"](https://didone.eu/) and, consequently,
it is specialized in 18th-century Italian opera arias. However, it is also prepared to work with other repertoires.

This tutorial is an introduction for people who are not experts in programming. If you are already an expert, just skip to the [Data Section](#data) and then go to the [Advanced Tutorial](https://musif.didone.eu/Tutorial%20poprock.html).


## Installation

First, you should install [`Python`](https://www.python.org/downloads/) > 3.10. An easy way to do this is by using [`Anaconda`](https://www.anaconda.com/products/distribution), especially if you are not used to commandline interface.A
Once you have installed `anaconda`:
1. Launch the `anaconda-navigator`
2. [Create an environment](https://docs.anaconda.com/navigator/getting-started/#managing-environments) selecting python version >= 3.10
3. Switch to the newly created environment by clicking on its name


To install `musif`:
1. [Download](https://raw.githubusercontent.com/DIDONEproject/musif/main/docs/source/Tutorial.ipynb) this notebook.
2. Start `jupyter` in your Anaconda environment.
3. Open this tutorial.
4. Run the following cell by clicking on it and pressing Ctrl+Enter.

In [ ]:
!pip install musif
# !pip install music21==9.1

In [ ]:
import os
import subprocess
import sys
from math import floor
from os import path
from pathlib import Path

from music21.stream.base import Measure, Score

import musif.extract.constants as C
from musif.extract.extract import parse_filename
from musif.logs import perr, pinfo, pwarn

sys.path.append(os.path.abspath('.'))
from feature_extraction.custom_conf import CustomConf
from feature_extraction.utils import get_ariaid

In [ ]:
def cut_by_measures_theme_A(cfg, data):
    score_id = get_ariaid(path.basename(data[C.DATA_FILE]))
    score: Score = data[C.DATA_SCORE]
    last_measure = 1000000
    for d in cfg.scores_metadata[last_measure]:
        if d["Id"] == score_id:
            last_measure = floor(float(d.get(cfg.end_of_theme_a, last_measure)))
            if last_measure == 0:
                name = data['file'].name
                pwarn(f'Last measure for {name} fil was found to be 0! Remember to update metadata before extraction ;) Setting last measure to the end of the score.\n')
                last_measure = 1000000
            break

    remove_everything_after_measure(score, last_measure)

def remove_everything_after_measure(score, last_measure):
    for part in score.parts:
        read_measures = 0
        elements_to_remove = []
        for measure in part.getElementsByClass(Measure):
            read_measures += 1
            if read_measures > last_measure:
                elements_to_remove.append(measure)
        part.remove(targetOrList=elements_to_remove)

def save_xml(data, new_filename):
    new_filename = str(new_filename) + '.xml'
    data[C.DATA_SCORE].write('musicxml', fp=f'{new_filename}')
    
def save_to_midi(filename):
    filename = str(filename)
    new_filename = filename + '.mid'
    if path.exists(new_filename):
            pinfo(f"{filename} already exists as MIDI, skipping it!")
            return
    cmd = ["mscore", "-fo", new_filename, filename + '.xml']
    pinfo(f"Converting {filename} to MIDI")
    try:
        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            timeout=120,
        )
    except subprocess.TimeoutExpired:
        pwarn(
            f"Continuing because time expired for file {filename}! Try running:\n"
            + "".join(cmd)
        ) 

In [ ]:
didone_config = "feature_extraction/config_extraction.yml"

cfg = CustomConf(didone_config)
data_path = 'data/xml/'
data_path_cutted = Path('data/xml/cutted_themeA/')


In [ ]:
for filename in sorted(Path(data_path).glob(f"*.xml")):
        data = {}
        new_filename = data_path_cutted / Path(filename.stem + '_cutted')
        if path.exists(str(new_filename) + '.xml'):
            pinfo(f"{filename} already exists as cutted xml, skipping it!")
            continue
        score = parse_filename(
            filename,
            cfg.split_keywords,
            expand_repeats=cfg.expand_repeats,
            export_dfs_to=None,
            remove_unpitched_objects=cfg.remove_unpitched_objects,
        )
        data[C.DATA_SCORE] = score
        data[C.DATA_FILE] = filename
        cut_by_measures_theme_A(cfg, data)
        data_path_cutted.mkdir(exist_ok=True)
        try:        
            save_xml(data, new_filename)
        except Exception as e:
            perr(f'There was an error saving score {filename} to xml: {e}. Skipping it!')
            continue
        try:        
            save_to_midi(new_filename)
        except Exception as e:
            perr(f'There was an error saving score {filename} to midi: {e}. Skipping it!')

